In [1]:
import tensorflow as tf
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from tqdm import tqdm
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import os
import importlib

dir_ml = 'C:/Users/Sten Stokroos/Desktop/NEW/zelf/Data/out'
randseed = 42
print("random seed: ", randseed)
np.random.seed(randseed)

def choose_data(dat, test_size, val_size, random_seed):
    if dat == 'ml2':
        train = pd.read_csv(os.path.join(dir_ml, 'ml_train2.csv'), sep="\t", header=None, names=['userId', 'songId', 'rating'], usecols=[0, 1, 2], engine="python")
        test = pd.read_csv(os.path.join(dir_ml, 'ml_test2.csv'), sep="\t", header=None, names=['userId', 'songId', 'rating'], usecols=[0, 1, 2], engine="python")
        train, val = train_test_split(ml_full, test_size=test_size, random_state=random_seed)

        user_ids = train['userId'].unique()
        item_ids = train['songId'].unique()
        n_users = len(user_ids)
        n_items = len(item_ids)
    elif dat == 'ml':
        ml_full = pd.read_csv(os.path.join(dir_ml, 'ml-1m_full.csv'), sep="\t", header=None, names=['userId', 'songId', 'rating'], usecols=[0, 1, 2], engine="python")
        user_ids = ml_full['userId'].unique()
        item_ids = ml_full['songId'].unique()
        n_users = len(user_ids)
        n_items = len(item_ids)
        train, test = train_test_split(ml_full, test_size=test_size, random_state=random_seed) #ADD VAL SETS FOR DATA SPLITS AS IN THESIS, ALSO IN THE GENERATIN PROCESS OF THE CONFOUDNERS 
    else:
        print('Wrong data input')
        return None, None, None, None, None

    print(f"Train set size: {train.shape[0]} ratings")
    print(f"Test set size: {test.shape[0]} ratings")
    return train, val, test, n_users, n_items


def load_confounders(dat, k, contype):
    CAUSEFIT_DIR = f'C:/Users/Sten Stokroos/Desktop/NEW/zelf/Data/exposure_output/{dat}_{contype}_exp_k_{k}.csv' #ADD FIXES TO MATCH DATA SPLITS
    conf_df = pd.read_csv(CAUSEFIT_DIR, header=None)
    confounder_data = conf_df.to_numpy().T
    return confounder_data

def load_data_rating(dat, columns=[0, 1, 2], sep="\t", include_validation=False, test_size=0.1, val_size=0.1, random_seed=None):
    train, val, test, n_users, n_items = choose_data(dat, test_size, val_size, random_seed)

    def build_matrix(df):
        rows = []
        cols = []
        ratings = []
        for line in df.itertuples():
            rows.append(line[1])
            cols.append(line[2])
            ratings.append(line[3])
        return csr_matrix((ratings, (rows, cols)), shape=(n_users, n_items)).todok()

    train_matrix = build_matrix(train)
    test_matrix = build_matrix(test)

    print("Load data finished. Number of users:", n_users, "Number of items:", n_items)
    return train_matrix, test_matrix, n_users, n_items


def run_model(module_name, class_name, k, contype='hpf', dat='ml', use_confounder=False, use_exposure=False, test_size=0.1, val_size=0.1, 
              hidden_neuron=500, learning_rate=0.001, reg_rate=0.1, epoch=20, batch_size=200, verbose=False, T=1, display_step=1000, save_path=None, num_runs=5):
    
    best_rmse_values = []

    for run in range(num_runs):
        random_seed = np.random.randint(10000)  # Generate a new random seed for each run
        print(f'random seed run {run}: {random_seed}')
        train, test, n_users, n_items = load_data_rating(dat, columns=[0, 1, 2], sep="\t", test_size=test_size, val_size=val_size, random_seed=random_seed)
        print(f"Loaded data: train={train.shape}, test={test.shape}, user={n_users}, item={n_items}")

        confounder_data = None
        exposure_data = None

        if use_confounder:
            confounder_data = load_confounders(dat, k, contype)
        if use_exposure:
            exposure_data = (train > 0).astype(np.float32).todense().T

        config = tf.compat.v1.ConfigProto()
        config.gpu_options.allow_growth = True

        with tf.compat.v1.Session(config=config) as sess:
            module = importlib.import_module(module_name)
            model_class = getattr(module, class_name)

            final_model = model_class(sess, n_users, n_items, learning_rate=learning_rate, reg_rate=reg_rate, epoch=epoch, batch_size=batch_size, verbose=verbose, T=T, display_step=display_step)
            final_model.build_network(hidden_neuron=hidden_neuron)
            
            if confounder_data is not None and exposure_data is not None:
                final_model.execute(train, test, confounder_data, exposure_data)
            elif confounder_data is not None:
                final_model.execute(train, test, confounder_data)
            else:
                final_model.execute(train, test)

            best_rmse_values.append(np.min(final_model.test_rmse_history))
            if save_path:
                class_folder = os.path.join(save_path, class_name.lower())
                os.makedirs(class_folder, exist_ok=True)
                np.save(os.path.join(class_folder, f'{contype}{epoch}_runssplit_{run}_train_loss_{dat}_k{k}.npy'), np.array(final_model.train_loss_history))
                np.save(os.path.join(class_folder, f'{contype}{epoch}_run_{run}_test_rmse_{dat}_k{k}.npy'), np.array(final_model.test_rmse_history))

    if save_path:
        class_folder = os.path.join(save_path, class_name.lower())
        np.save(os.path.join(class_folder, f'{contype}{epoch}_best_rmse_values_{dat}_k{k}.npy'), np.array(best_rmse_values))

    avg_best_rmse = np.mean(best_rmse_values)
    print(f"Average Best RMSE: {avg_best_rmse}")

    return best_rmse_values

    


random seed:  42


In [2]:
def run_all_models(result_path, models, k_values, contype):
    for c in contype:
        for k in k_values:
            for model in models:
                use_confounder = 'conf' in model['module_name']
                use_exposure = 'exp' in model['module_name']
                
                print(f"Running {model['class_name']} with k={k}, use_confounder={use_confounder}, use_exposure={use_exposure}")
                run_model(model['module_name'], model['class_name'], k, c, dat='ml2', use_confounder=use_confounder, use_exposure=use_exposure, test_size=0.1, val_size=0.1, hidden_neuron=500, 
                        learning_rate=0.001, reg_rate=1, epoch=200, batch_size=512, verbose=True, save_path=result_path, num_runs=5)
                
                print(f"Completed {model['class_name']} with k={k}")

result_path = 'C:/Users/Sten Stokroos/Desktop/NEW/zelf/results2'

models = [
    {'module_name': 'urec_og', 'class_name': 'UAutoRec'},
    {'module_name': 'urec_1_conf', 'class_name': 'UAutoRec1conf'},
    {'module_name': 'urec_1_confexp', 'class_name': 'UAutoRec1confexp'},
    {'module_name': 'urec_2_conf', 'class_name': 'UAutoRec2conf'},
    {'module_name': 'urec_2_confexp', 'class_name': 'UAutoRec2confexp'},
    {'module_name': 'urec_3_conf', 'class_name': 'UAutoRec3conf'},
    {'module_name': 'urec_3_confexp', 'class_name': 'UAutoRec3confexp'},
    # {'module_name': 'urec_4_conf', 'class_name': 'UAutoRec4conf'},
    # {'module_name': 'urec_4_confexp', 'class_name': 'UAutoRec4confexp'}
]

k_values = [32]
contype = ['hpf']

run_all_models(result_path, models, k_values, contype)



Running UAutoRec with k=32, use_confounder=False, use_exposure=False
random seed run 0: 7270
Train set size: 900191 ratings
Test set size: 100022 ratings
Load data finished. Number of users: 6040 Number of items: 3706
Loaded data: train=(6040, 3706), test=(6040, 3706), user=6040, item=3706
UAutoRec.


Training:   0%|          | 1/200 [13:58<46:21:56, 838.77s/epoch, Loss=3.22e+4, RMSE=0.9, MAE=0.715]


random seed run 1: 5364
Train set size: 900191 ratings
Test set size: 100022 ratings
Load data finished. Number of users: 6040 Number of items: 3706
Loaded data: train=(6040, 3706), test=(6040, 3706), user=6040, item=3706
UAutoRec.


Training:   0%|          | 1/200 [14:32<48:15:18, 872.96s/epoch, Loss=3.35e+4, RMSE=0.895, MAE=0.71]


random seed run 2: 3076
Train set size: 900191 ratings
Test set size: 100022 ratings
Load data finished. Number of users: 6040 Number of items: 3706
Loaded data: train=(6040, 3706), test=(6040, 3706), user=6040, item=3706
UAutoRec.


...

random seed run 3: 7204
Train set size: 900191 ratings
Test set size: 100022 ratings
Load data finished. Number of users: 6040 Number of items: 3706
Loaded data: train=(6040, 3706), test=(6040, 3706), user=6040, item=3706
UAutoRec with Confounder and Exposure.


Training: 100%|██████████| 200/200 [36:48<00:00, 11.04s/epoch, Loss=9.04e+3, RMSE=1.04, MAE=0.829]


random seed run 4: 1308
Train set size: 900191 ratings
Test set size: 100022 ratings
Load data finished. Number of users: 6040 Number of items: 3706
Loaded data: train=(6040, 3706), test=(6040, 3706), user=6040, item=3706
UAutoRec with Confounder and Exposure.


Training: 100%|██████████| 200/200 [36:41<00:00, 11.01s/epoch, Loss=9.16e+3, RMSE=1.04, MAE=0.828]


Average Best RMSE: 0.9890661437257361
Completed UAutoRec4confexp with k=32


In [4]:
import numpy as np
import pandas as pd
from scipy.stats import t
import os

def compute_statistics(data):
    """Compute mean, standard deviation, and sample size using .describe() method."""
    desc = pd.DataFrame(data).describe()
    mean = desc.loc['mean'].values
    std = desc.loc['std'].values
    n = desc.loc['count'].values
    return mean, std, n

def compute_confidence_intervals(mean, std, n, confidence_level=0.95):
    t_value = t.ppf((1 + confidence_level) / 2., n-1)
    margin_of_error = t_value * (std / np.sqrt(n))
    lower_bound = mean - margin_of_error
    upper_bound = mean + margin_of_error
    return lower_bound, upper_bound

def load_rmse_values(result_path, models, k_values, contype, epoch, dat):
    results = []

    for model in models:
        class_name = model['class_name'].lower()
        for k in k_values:
            rmse_file = os.path.join(result_path, class_name, f'{contype}{epoch}_best_rmse_values_{dat}_k{k}.npy')
            if os.path.exists(rmse_file):
                rmse_values = np.load(rmse_file)
                mean, std, n = compute_statistics(rmse_values)
                lower_bound, upper_bound = compute_confidence_intervals(mean, std, n)
                results.append({
                    'Model': model['class_name'],
                    'k': k,
                    'Confounder': contype,
                    'Mean Best RMSE': mean[0],
                    'CI Lower': lower_bound[0],
                    'CI Upper': upper_bound[0]
                })

    results_df = pd.DataFrame(results)
    print(results_df)
    return results_df

result_path = 'C:/Users/Sten Stokroos/Desktop/NEW/zelf/results2'
models = [
    {'module_name': 'urec_og', 'class_name': 'UAutoRec'},
    {'module_name': 'urec_1_conf', 'class_name': 'UAutoRec1conf'},
    {'module_name': 'urec_1_confexp', 'class_name': 'UAutoRec1confexp'},
    {'module_name': 'urec_2_conf', 'class_name': 'UAutoRec2conf'},
    {'module_name': 'urec_2_confexp', 'class_name': 'UAutoRec2confexp'},
    {'module_name': 'urec_3_conf', 'class_name': 'UAutoRec3conf'},
    {'module_name': 'urec_3_confexp', 'class_name': 'UAutoRec3confexp'},
    # {'module_name': 'urec_4_conf', 'class_name': 'UAutoRec4conf'},
    # {'module_name': 'urec_4_confexp', 'class_name': 'UAutoRec4confexp'}
]
k_values = [32]
contype = ['hpf', 'mlp']
epoch = 200
dat = 'ml2'

results_df = load_rmse_values(result_path, models, k_values, contype, epoch, dat)


Empty DataFrame
Columns: []
Index: []


In [6]:
import numpy as np
import pandas as pd
from scipy.stats import t
import os
import re

def compute_statistics(data):
    """Compute mean, standard deviation, and sample size using .describe() method."""
    desc = pd.DataFrame(data).describe()
    mean = desc.loc['mean'].values
    std = desc.loc['std'].values
    n = desc.loc['count'].values
    return mean, std, n

def compute_confidence_intervals(mean, std, n, confidence_level=0.95):
    t_value = t.ppf((1 + confidence_level) / 2., n-1)
    margin_of_error = t_value * (std / np.sqrt(n))
    lower_bound = mean - margin_of_error
    upper_bound = mean + margin_of_error
    return lower_bound, upper_bound

def load_rmse_values(result_path, models, k_values, contype, epoch, dat):
    results = []

    for model in models:
        class_name = model['class_name'].lower()
        for k in k_values:
            for c in contype:
                best_rmse_values = []
                pattern = re.compile(f'{c}{epoch}_run_\\d+_test_rmse_{dat}_k{k}.npy')
                model_folder = os.path.join(result_path, class_name)
                for file in os.listdir(model_folder):
                    if pattern.match(file):
                        file_path = os.path.join(model_folder, file)
                        rmse_values = np.load(file_path)
                        best_rmse_values.append(np.min(rmse_values))
                
                if best_rmse_values:
                    mean, std, n = compute_statistics(best_rmse_values)
                    lower_bound, upper_bound = compute_confidence_intervals(mean, std, n)
                    results.append({
                        'Model': model['class_name'],
                        'Confounder': c,
                        'Mean Best RMSE': mean[0],
                        'CI Lower': lower_bound[0],
                        'CI Upper': upper_bound[0]
                    })

    results_df = pd.DataFrame(results)
    print(results_df)
    return results_df

result_path = 'C:/Users/Sten Stokroos/Desktop/NEW/zelf/results2'
models = [
    {'module_name': 'urec_og', 'class_name': 'UAutoRec'},
    {'module_name': 'urec_1_conf', 'class_name': 'UAutoRec1conf'},
    {'module_name': 'urec_1_confexp', 'class_name': 'UAutoRec1confexp'},
    {'module_name': 'urec_2_conf', 'class_name': 'UAutoRec2conf'},
    {'module_name': 'urec_2_confexp', 'class_name': 'UAutoRec2confexp'},
    {'module_name': 'urec_3_conf', 'class_name': 'UAutoRec3conf'},
    {'module_name': 'urec_3_confexp', 'class_name': 'UAutoRec3confexp'},
    # {'module_name': 'urec_4_conf', 'class_name': 'UAutoRec4conf'},
    # {'module_name': 'urec_4_confexp', 'class_name': 'UAutoRec4confexp'}
]
k_values = [32]
contype = ['hpf', 'mlp']
epoch = 200
dat = 'ml2'

results_df = load_rmse_values(result_path, models, k_values, contype, epoch, dat)


               Model Confounder  Mean Best RMSE  CI Lower  CI Upper
0           UAutoRec        hpf        0.937451  0.934960  0.939942
1           UAutoRec        mlp        0.937032  0.933485  0.940579
2      UAutoRec1conf        hpf        0.935591  0.934503  0.936679
3      UAutoRec1conf        mlp        0.936585  0.935669  0.937501
4   UAutoRec1confexp        hpf        0.941072  0.940239  0.941905
5   UAutoRec1confexp        mlp        0.940575  0.939456  0.941694
6      UAutoRec2conf        hpf        0.926922  0.924554  0.929289
7      UAutoRec2conf        mlp        0.927007  0.925342  0.928673
8   UAutoRec2confexp        hpf        0.921681  0.919984  0.923378
9   UAutoRec2confexp        mlp        0.921965  0.920010  0.923921
10     UAutoRec3conf        hpf        0.935299  0.932746  0.937853
11     UAutoRec3conf        mlp        0.934173  0.931090  0.937256
12  UAutoRec3confexp        hpf        0.930171  0.927538  0.932803
13  UAutoRec3confexp        mlp        0.931885 